In [1]:
from math import pow
import random
import time
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def backward_substitution(L, b):
    x = np.zeros_like(b)
    for i in range(len(L)-1, -1, -1):
        s = 0
        for j in range (i+1, len(L)):
            s += A[i, j]*x[j]
        x[i] = (b[i] - s)/A[i, i]
    return x

In [3]:
A = np.array([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
b = [2, 4, 3]
print(backward_substitution(A, b))
print(np.linalg.solve(A, b))

[2 2 1]
[2. 2. 1.]


In [4]:
def forward_substitution(L, b):
    x = np.zeros_like(b)
    for i in range(len(L)):
        s = 0
        for j in range(i):
            s += L[i, j]*x[j]
        x[i] = (b[i]-s)/L[i, i]
    return x

In [11]:
A = np.array([[1, 0, 0], [-1, 2, 0], [0, -3, 4]])
b = [2, 4, 3]
print(forward_substitution(A, b))
print(np.linalg.solve(A, b))

[2 3 3]
[2. 3. 3.]


In [12]:
def random_lower_triangular_matrix(m, n, random_range = (1, 10)):
    A = []
    for i in range(m):
        wiersz = []
        for j in range(n):
            if i<=j:
                wiersz.append(random.randint(random_range[0], random_range[1]))
            else:
                wiersz.append(0)
        A.append(wiersz)
    return np.array(A)

In [28]:
def gaussian_elimination(A, b):
    n = A.shape[0]
    A_b = np.c_[A, b]
    for i in range(n):
        ilosc_translacji = 0
        if abs(A_b[i, i])<1/(2**16):
            print('weszlo dla i=', i)
            ilosc_translacji += 1
            for k in range(i+1, n):
                if abs(A_b[k, i])>=1/(2**16):
                    A_b[[k, i]] = A_b[[i, k]]
                    pass
        print(' dalej dla i=', i)
        for j in range(i+1, n):
            const = -A_b[j, i]/A_b[i, i]
            A_b[j, i:] = A_b[j, i:] + const*A_b[i, i:]
            
            
    return A_b[:,:n-1], A_b[:, n]

In [31]:
A = np.array([[1, 0, 0], [-1, 2, 0], [0, -3, 4]])
A = np.array([[0, 1, 2, 4], [0, 2, 1, 1], [-1, 3, 1, 1], [0, 4, 1, 2]])
b = np.array([1, 2, 3, 4])
U, c = gaussian_elimination(A, b)
print('U=', U, '\nc=', c)

weszlo dla i= 0
 dalej dla i= 0
 dalej dla i= 1
 dalej dla i= 2
 dalej dla i= 3
U= [[-1  3  1]
 [ 0  2  1]
 [ 0  0  1]
 [ 0  0  0]] 
c= [3 2 0 0]
